In [2]:
# !pip install wikipedia

In [2]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

from langchain_core.tools import tool

load_dotenv()

True

True

In [5]:
# 1. create the tool
@tool
def get_temperature(city: str) -> str:
    """
    To find the current temperature in the given city
    
    Args:
        city: the name of the city that we want to have its temperature.
    """
    return "15 celcius"

@tool
def get_time(city: str) -> str:
    """
    To find the time in the given city
    
    Args:
        city: the name of the city that we want to have its time.
    """
    return "12:30:02"

In [6]:
model = ChatOpenAI(model='gpt-3.5-turbo')

# 2. to connect the tool with the model
tools = [get_temperature, get_time]
model_with_tool = model.bind_tools(tools)

In [7]:
# 3. execution of the tool (if needed)
prompt_template = "What is the time in {city}? and also give me its current temperature."
prompt = PromptTemplate.from_template(prompt_template).invoke({'city': "California"})
print(prompt)

result_with_tool = model_with_tool.invoke(prompt)

text='What is the time in California? and also give me its current temperature.'


In [8]:
result_with_tool.content

''

__As it can be seen, LLM does not return any answer__

    But if we use `tool_calls` we can see that the model wants us to call `get_time` and `get_temperature` tools.

In [9]:
print(result_with_tool.tool_calls)

[{'name': 'get_time', 'args': {'city': 'California'}, 'id': 'call_TXREKOT8bA6lxVWpusCSMLZn', 'type': 'tool_call'}, {'name': 'get_temperature', 'args': {'city': 'California'}, 'id': 'call_Eu8KAxngwVaKKki100u2AGIQ', 'type': 'tool_call'}]


Now we can use this information to create a prompt for the LLM with the needed information.

In [10]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(prompt.text)]
messages.append(result_with_tool)

tools_map = {"get_temperature": get_temperature, "get_time": get_time}
for tool_call in result_with_tool.tool_calls:
    selected_tool = tools_map[tool_call['name']]
    tool_output = selected_tool.invoke(tool_call['args'])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call['id']))
    
print(messages)

[HumanMessage(content='What is the time in California? and also give me its current temperature.', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TXREKOT8bA6lxVWpusCSMLZn', 'function': {'arguments': '{"city": "California"}', 'name': 'get_time'}, 'type': 'function'}, {'id': 'call_Eu8KAxngwVaKKki100u2AGIQ', 'function': {'arguments': '{"city": "California"}', 'name': 'get_temperature'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 127, 'total_tokens': 170, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8a230188-44c6-41d1-8643-37fa8245618e-0', tool_calls=[{'name': 'get_time', '

In [11]:
final_result = model_with_tool.invoke(messages)
final_result.content

'The current time in California is 12:30:02 and the temperature is 15°C.'

## Wikipedia

In [4]:
from wikipedia_utils import get_wikipedia_text

In [10]:
# Example usage:
name = "Albert Einstein"
print(get_person_details(name))


NameError: name 'get_person_details' is not defined